In [20]:
import pandas as pd
from datetime import datetime

In [2]:
def clean_vic_df(df):
    
    # remove cases, active cases, daily recovered
    # reorder columns
    # retain only dates from 1/1/22 to 4/4/22
    # add cases_female and cases_male, into new cases column
    # remove cases_female and cases_male
    
    pass

In [85]:
df = pd.read_csv('sources/vic0-9.csv')
df.head()

,agegroup,Cases_Male,Active_Cases,Cases,Daily_Recovered,Date,Sex,Cases_Female
0,0-9,0.0,0.0,0.0,0.0,1/25/2020,Female,0.0
1,0-9,0.0,0.0,0.0,0.0,1/25/2020,Male,0.0
2,0-9,0.0,0.0,0.0,0.0,1/26/2020,Female,0.0
3,0-9,0.0,0.0,0.0,0.0,1/26/2020,Male,0.0
4,0-9,0.0,0.0,0.0,0.0,1/27/2020,Female,0.0


In [86]:
# drop unneeded columns
df.drop(['Active_Cases', 'Daily_Recovered', 'Cases', 'Sex'], axis=1, inplace=True)
df.head()

,agegroup,Cases_Male,Date,Cases_Female
0,0-9,0.0,1/25/2020,0.0
1,0-9,0.0,1/25/2020,0.0
2,0-9,0.0,1/26/2020,0.0
3,0-9,0.0,1/26/2020,0.0
4,0-9,0.0,1/27/2020,0.0


In [87]:
# reorder columns
df = df[['Date', 'agegroup', 'Cases_Male', 'Cases_Female']]
df.head()

,Date,agegroup,Cases_Male,Cases_Female
0,1/25/2020,0-9,0.0,0.0
1,1/25/2020,0-9,0.0,0.0
2,1/26/2020,0-9,0.0,0.0
3,1/26/2020,0-9,0.0,0.0
4,1/27/2020,0-9,0.0,0.0


In [88]:
# convert date strings to python datetime objects
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

Date            datetime64[ns]
agegroup                object
Cases_Male             float64
Cases_Female           float64
dtype: object

In [89]:
# retain only dates from 1/1/22 to 4/4/22
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 4, 4)

df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
print(f'Rows = {df.shape[0]}')

print(df.head())
print(df.tail())

Rows = 188
          Date agegroup  Cases_Male  Cases_Female
740 2022-01-01      0-9         0.0         188.0
741 2022-01-02      0-9         0.0         219.0
742 2022-01-03      0-9         0.0         325.0
743 2022-01-04      0-9         0.0         386.0
744 2022-01-05      0-9         0.0         391.0
           Date agegroup  Cases_Male  Cases_Female
1604 2022-03-31      0-9       583.0           0.0
1605 2022-04-01      0-9       526.0           0.0
1606 2022-04-02      0-9       507.0           0.0
1607 2022-04-03      0-9       610.0           0.0
1608 2022-04-04      0-9       739.0           0.0


In [90]:
df

,Date,agegroup,Cases_Male,Cases_Female
740,2022-01-01,0-9,0.0,188.0
741,2022-01-02,0-9,0.0,219.0
742,2022-01-03,0-9,0.0,325.0
743,2022-01-04,0-9,0.0,386.0
744,2022-01-05,0-9,0.0,391.0
...,...,...,...,...
1604,2022-03-31,0-9,583.0,0.0
1605,2022-04-01,0-9,526.0,0.0
1606,2022-04-02,0-9,507.0,0.0
1607,2022-04-03,0-9,610.0,0.0


In [91]:
# combine duplicate dates; add male and female cases
df = df.groupby('Date').sum()
df['Cases'] = df['Cases_Male'] + df['Cases_Female']

print(f'Rows = {df.shape[0]}')
print(df.head())
print(df.tail())

Rows = 94
            Cases_Male  Cases_Female  Cases
Date                                       
2022-01-01       201.0         188.0  389.0
2022-01-02       240.0         219.0  459.0
2022-01-03       329.0         325.0  654.0
2022-01-04       419.0         386.0  805.0
2022-01-05       385.0         391.0  776.0
            Cases_Male  Cases_Female   Cases
Date                                        
2022-03-31       583.0         550.0  1133.0
2022-04-01       526.0         480.0  1006.0
2022-04-02       507.0         492.0   999.0
2022-04-03       610.0         559.0  1169.0
2022-04-04       739.0         731.0  1470.0


In [92]:
# drop male and female cases; no longer needed
df.drop(['Cases_Male', 'Cases_Female'], axis=1, inplace=True)

In [93]:
df.columns = ['0-9']

In [98]:
df

,0-9
Date,
2022-01-01,389.0
2022-01-02,459.0
2022-01-03,654.0
2022-01-04,805.0
2022-01-05,776.0
...,...
2022-03-31,1133.0
2022-04-01,1006.0
2022-04-02,999.0
